In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import os
from os.path import join
from ast import literal_eval
import itertools
from urllib.parse import urlparse
from tqdm.notebook import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from tensorboard import notebook
import time

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
from transformers import BertModel
bert_id = "google/bert_uncased_L-2_H-128_A-2"

# 1 Load Data

In [2]:
train_joint = pd.read_hdf(join("preprocessed_data","train_joint.h5"), key="s")
validation_join = pd.read_hdf(join("preprocessed_data","validation_joint.h5"), key="s")

In [3]:
train_joint

is_relevant sector_ids  sentence_position  \
doc_id sentence_id                                              
51657  0                      0         []           0.000000   
       1                      0         []           0.693147   
       2                      0         []           1.098612   
       3                      0         []           1.386294   
       4                      0         []           1.609438   
...                         ...        ...                ...   
34512  121                    0         []           4.804021   
       122                    0         []           4.812184   
       123                    0         []           4.820282   
       124                    0         []           4.828314   
       125                    0         []           4.836282   

                    sentence_length  \
doc_id sentence_id                    
51657  0                   5.252273   
       1                   4.127134   
       2                   4.682131   
       3                   4.875197   
       4                   4.204693   
...                             ...   
34512  121                 4.234107   
       122                 5.017280   
       123                 4.234107   
       124                 4.174387   
       125                 5.420535   

                                                   tokenized_sentence  \
doc_id sentence_id                                                      
51657  0            [101, 2047, 4341, 2937, 3360, 2415, 1999, 2474...   
       1            [101, 10110, 2003, 2012, 1996, 2415, 1997, 199...   
       2            [101, 1996, 2047, 3360, 2415, 2038, 2366, 2062...   
       3            [101, 1996, 4341, 2937, 3360, 2415, 2001, 2764...   
       4            [101, 2116, 1997, 2122, 3360, 2272, 2013, 3532...   
...                                                               ...   
34512  121          [101, 2174, 1010, 11470, 19621, 2015, 2024, 20...   
       122          [101, 1999, 5712, 1010, 2045, 2024, 4311, 1997...   
       123          [101, 1996, 9353, 9331, 2015, 2136, 2097, 2562...   
       124          [101, 2017, 2064, 2424, 2019, 19184, 1997, 203...   
       125          [101, 2340, 9353, 9331, 2015, 11470, 3229, 191...   

                    project_name  country_code  url  text_length  \
doc_id sentence_id                                                 
51657  0                       1             2  139     7.699389   
       1                       1             2  139     7.699389   
       2                       1             2  139     7.699389   
       3                       1             2  139     7.699389   
       4                       1             2  139     7.699389   
...                          ...           ...  ...          ...   
34512  121                     5             1    0    10.068493   
       122                     5             1    0    10.068493   
       123                     5             1    0    10.068493   
       124                     5             1    0    10.068493   
       125                     5             1    0    10.068493   

                    sentence_count  
doc_id sentence_id                  
51657  0                  2.944439  
       1                  2.944439  
       2                  2.944439  
       3                  2.944439  
       4                  2.944439  
...                            ...  
34512  121                4.836282  
       122                4.836282  
       123                4.836282  
       124                4.836282  
       125                4.836282  

[261981 rows x 10 columns]

# 2 Define Dataset

In [4]:
class IsRelevantDataset(Dataset):
    def __init__(self, joint_dataframe: pd.DataFrame, device=device, dimensions = None):
        self.X = joint_dataframe[["sentence_position", "sentence_length", "tokenized_sentence", "project_name", "country_code", "url", "text_length", "sentence_count"]].to_numpy()
        self.Y = joint_dataframe["is_relevant"].to_numpy()
        self.device = device
        
        if dimensions is None:
            self.dimensions = ((1, (4, len(set(self.X[:,3])), len(set(self.X[:,4])), len(set(self.X[:,5])))), 2)
        else:
            self.dimensions = dimensions
        
    def __len__(self):
        return len(self.Y)

    
    def __getitem__(self, idx, x_one_hot = True, x_train_ready = True):
        
        """
        Note that x_train_ready implies x_one_hot
        """
        x_tmp = self.X[idx]
        metric_x = torch.tensor([x_tmp[0], x_tmp[1], x_tmp[6], x_tmp[7]], device=self.device)#numerical features
        sentence_x = torch.tensor(x_tmp[2], device=self.device, dtype=torch.long)#bert features
        sentence_x = torch.cat((sentence_x, torch.zeros(512 - sentence_x.shape[0], device=self.device, dtype= torch.long)))
        
        #one hot features:
        project_name_x = torch.tensor(x_tmp[3], device=self.device, dtype=torch.long)
        country_code_x = torch.tensor(x_tmp[4], device=self.device, dtype=torch.long)
        url_x = torch.tensor(x_tmp[5], device=self.device)
        
        y = torch.tensor(self.Y[idx], device=self.device, dtype=torch.long)

        if x_train_ready or x_one_hot:
            project_name_x = nn.functional.one_hot(project_name_x, num_classes = self.dimensions[0][1][1])
            country_code_x = nn.functional.one_hot(country_code_x, num_classes = self.dimensions[0][1][2])
            url_x = nn.functional.one_hot(url_x, num_classes = self.dimensions[0][1][3])
        if x_train_ready:
            x_other = torch.cat((metric_x, project_name_x, country_code_x, url_x), dim=0)
            return (sentence_x, x_other), y
        
        return (sentence_x, (metric_x, project_name_x, country_code_x, url_x)), y

In [5]:
train_ds = IsRelevantDataset(train_joint, device = device)
validation_ds = IsRelevantDataset(validation_join, device = device, dimensions = train_ds.dimensions)

# 4 Model Definition

In [63]:
class IsRelevantNet(nn.Module):
    def __init__(self, bert: BertModel, input_size, output_size):
        super(IsRelevantNet, self).__init__()
        
        self.bert = bert
        self.linear_after_bert = nn.Linear(bert.config.hidden_size, 256)
        self.feed_forward = nn.Sequential(
            #nn.BatchNorm1d(bert.config.hidden_size + input_size),#just a feeling this might be nice
            nn.Linear(256 + input_size, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        x_bert = x[0]
        x_other = x[1]
        y_bert = self.bert(x[0])["last_hidden_state"][:,0] #all batches but only clf output
        y_bert = self.linear_after_bert(y_bert)
        x = torch.cat((y_bert, x_other), dim=1)#dim=1 is feature dimensions (0 is batch dim)
        
        return self.feed_forward(x)

# 5 Training Routine

In [64]:
def update(model, optimizer, loss, loader, logger , i):
    model.train()
    for (x, y) in tqdm(loader):
        i+=1
        y_hat = model(x)
        l = loss(y_hat, y)
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        if logger:
            logger.add_scalar("loss",l.detach().cpu(),i)
    

@torch.no_grad()
def evaluate(model, metric, loader, logger = None):
    model.eval()
    
    for x,y in loader:
        y_hat = model(x)
        l = metric(y_hat, y)
        if logger:
            logger.next_eval(l)
            
    if logger:
        logger.submit_eval()

In [65]:
lr = 1e-4
batch_size=16
epochs = 10
bert_hidden = 768

In [66]:
train_dl = DataLoader(train_ds,batch_size  = batch_size, shuffle=True)
validation_dl = DataLoader(train_ds, batch_size  = 64, shuffle=False)

loss = nn.CrossEntropyLoss()
model = IsRelevantNet(BertModel.from_pretrained(bert_id).to(device), sum(train_ds.dimensions[0][1]), train_ds.dimensions[1]).to(device)
#Should not train bert (for now)
#model.bert.train(False)
#for p in model.bert.parameters():
#    p.requires_grad = False
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

print(f"Number of parameters (including bert): {sum(p.numel() for p in model.parameters())}")
print(f"Number of trainable parameters (excluding bert): {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Number of parameters (including bert): 5132098
Number of trainable parameters (excluding bert): 5132098


In [67]:
print(len(train_dl))

16374


In [68]:
base_logdir = "logs"
logdir = join(base_logdir, "log6")
tensorboard_port = 6006
sw = SummaryWriter(logdir)

!start tensorboard --reload_multifile true --port $tensorboard_port --logdir $logdir
time.sleep(8)
notebook.display(port=tensorboard_port, height=800)

Selecting TensorBoard with logdir log6 (started 0:00:04 ago; port 6006, pid 16864).


In [69]:
for n_epoch in range(1, epochs+1):
    update(model, optimizer, loss, train_dl, sw, (n_epoch-1)*len(train_dl))

  0%|          | 0/16374 [00:00<?, ?it/s]

  0%|          | 0/16374 [00:00<?, ?it/s]

  0%|          | 0/16374 [00:00<?, ?it/s]

  0%|          | 0/16374 [00:00<?, ?it/s]

  0%|          | 0/16374 [00:00<?, ?it/s]

KeyboardInterrupt: 